In [4]:
import pandas as pd
import numpy as np


In [5]:
raw = pd.read_csv("Absenteeism_data.csv")
raw.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [6]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         700 non-null    int64  
 1   Reason for Absence         700 non-null    int64  
 2   Date                       700 non-null    object 
 3   Transportation Expense     700 non-null    int64  
 4   Distance to Work           700 non-null    int64  
 5   Age                        700 non-null    int64  
 6   Daily Work Load Average    700 non-null    float64
 7   Body Mass Index            700 non-null    int64  
 8   Education                  700 non-null    int64  
 9   Children                   700 non-null    int64  
 10  Pets                       700 non-null    int64  
 11  Absenteeism Time in Hours  700 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


In [7]:
raw["reason_group"] = pd.cut(
    x=raw["Reason for Absence"],
    bins=[-np.inf, 0, 14, 17, 21, np.inf],
    labels=["unknown", "sickness", "pregnancy", "accident", "other"],

)
 
reason_dummy = pd.get_dummies(raw["reason_group"], prefix="reason_group", drop_first=True).head(10)

In [8]:
df = pd.concat([raw, reason_dummy], axis=1)
#df.head(10)

In [9]:
df['timestamp'] = pd.to_datetime(df['Date'], dayfirst=True)
type(df['timestamp'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [10]:
df['month'] = df['timestamp'].dt.month
df['day_of_week'] = df['timestamp'].dt.day_of_week

In [11]:
df['higher_education'] = df['Education'].apply(lambda x: 1 if x > 1 else 0)


In [12]:
#target Einstufung
# der vorteil, der median zu benutyen ist, dass die Daten ausgleich geteilt werden 
benchmark = df['Absenteeism Time in Hours'].median()
df['target'] = df['Absenteeism Time in Hours'].apply(lambda x: 1 if x > benchmark else 0)
df.target.sum()/df.target.shape[0] # für logisticregression 40/60 aufgeteilt Daten funktioniert gut, aber für Neuralnetwork 50/50 ist besser

0.45571428571428574

In [13]:
df_select = df.copy()
df_select.drop(columns=['Date', 'ID', 'Reason for Absence', 'Absenteeism Time in Hours', 'reason_group', 'timestamp', 'Education'], inplace=True)
input = df_select.iloc[:, :-1]


In [20]:
#die Daten satnardisieren
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
input_scaled = scaler.fit_transform(input)


In [ ]:
#daten in train und test aufteilen und mischen
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input_scaled, df_select['target'], test_size=0.2, random_state=42, stratify=df_select['target'], shuffle=True)